<a href="https://colab.research.google.com/github/mille055/Rec_Project/blob/main/notebooks/getting_more_podcast_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install app_store_scraper unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 12.9 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from app_store_scraper import AppStore
import json
import re
import unidecode
import time
from tqdm import tqdm
import pickle

In [6]:

!git clone 'https://github.com/mille055/Rec_Project.git'

Cloning into 'Rec_Project'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 181 (delta 29), reused 34 (delta 8), pack-reused 112
Receiving objects: 100% (181/181), 45.13 MiB | 19.53 MiB/s, done.
Resolving deltas: 100% (86/86), done.
Updating files: 100% (18/18), done.


In [7]:
myscraped = pd.read_pickle('/content/Rec_Project/data/cmm_scraped_podcasts_with_ratings.pkl')
myscraped

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,itunes_id,episode_descriptions,user,rating
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,RobinFerris,5.0
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,Pops.99,1.0
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,ReddEye81,5.0
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,Keyta7777,2.0
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,Okkupent,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,Monijansand,5.0
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,trinityangel13,5.0
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,Kweenkeys,5.0
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,JoyfulJoyfulWOG,5.0


In [21]:
set_of_links = set(myscraped.link.tolist())
set_of_links

{'https://podcasts.apple.com/us/podcast/constant-wonder/id1436541194',
 'https://podcasts.apple.com/us/podcast/american-metamorphosis/id1553973947',
 'https://podcasts.apple.com/us/podcast/global-news-podcast/id135067274',
 'https://podcasts.apple.com/us/podcast/turning-to-the-mystics-with-james-finley/id1494041647',
 'https://podcasts.apple.com/us/podcast/somewhere-in-the-skies/id1227858637',
 'https://podcasts.apple.com/us/podcast/knowing-faith/id1274228164',
 'https://podcasts.apple.com/us/podcast/the-mallory-bros-podcast/id1520856346',
 'https://podcasts.apple.com/us/podcast/the-secret-history-of-hollywood/id965023632',
 'https://podcasts.apple.com/us/podcast/cryptids-of-the-corn/id1599884695',
 'https://podcasts.apple.com/us/podcast/podcast-unlocked/id276268454',
 'https://podcasts.apple.com/us/podcast/ride-the-lightning-tesla-motors-unofficial-podcast/id1025399574',
 'https://podcasts.apple.com/us/podcast/confessions-of-a-stenographer/id1489688908',
 'https://podcasts.apple.com/u

In [11]:
country = "us"
num_pages = 10  # Adjust this to fetch more or fewer pages of reviews

In [28]:
def fetch_individual_ratings(link, country = 'us', num_pages=10):
    app_name = get_app_name(link)
    print('app name is ', app_name)
    podcast_id = get_podcast_id(link)
    print('podcast_id is', podcast_id)
    podcast = AppStore(country=country, app_name = app_name, app_id=podcast_id)
    podcast.review(how_many=num_pages)
    reviews = podcast.reviews
    individual_ratings = []

    for review in reviews:
        individual_ratings.append({
            'itunes_id': podcast_id,
            'user': review['userName'],
            'rating': review['rating'],
            'title': review['title'],
            'review': review['review']
        })

    return individual_ratings


def get_app_name(url):
    app_name = url.split("/id")[-2]
    return app_name


def get_podcast_id(url):
    podcast_id = url.split("/id")[-1].split("?")[0]
    return podcast_id

In [29]:
test = ['https://podcasts.apple.com/us/podcast/hbos-the-last-of-us-podcast/id1660320068',
 'https://podcasts.apple.com/us/podcast/this-is-uncomfortable/id1466681015',
 'https://podcasts.apple.com/us/podcast/thomas-friends-storytime-us/id1506815544',
 'https://podcasts.apple.com/us/podcast/ali-on-the-run-show/id1204523870']

In [ ]:
ratings = []
for item in set_of_links:
  rating = fetch_individual_ratings(item)
  ratings.append(rating)

app name is  https://podcasts.apple.com/us/podcast/constant-wonder
podcast_id is 1436541194
app name is  https://podcasts.apple.com/us/podcast/american-metamorphosis
podcast_id is 1553973947
app name is  https://podcasts.apple.com/us/podcast/global-news-podcast
podcast_id is 135067274
app name is  https://podcasts.apple.com/us/podcast/turning-to-the-mystics-with-james-finley
podcast_id is 1494041647
app name is  https://podcasts.apple.com/us/podcast/somewhere-in-the-skies
podcast_id is 1227858637
app name is  https://podcasts.apple.com/us/podcast/knowing-faith
podcast_id is 1274228164
app name is  https://podcasts.apple.com/us/podcast/the-mallory-bros-podcast
podcast_id is 1520856346
app name is  https://podcasts.apple.com/us/podcast/the-secret-history-of-hollywood
podcast_id is 965023632
app name is  https://podcasts.apple.com/us/podcast/cryptids-of-the-corn
podcast_id is 1599884695
app name is  https://podcasts.apple.com/us/podcast/podcast-unlocked
podcast_id is 276268454
app name is

ERROR:Base:Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url: /v1/catalog/us/apps/1680169269/reviews?l=en-GB&offset=0&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 404 error responses'))


app name is  https://podcasts.apple.com/us/podcast/the-mismatch
podcast_id is 1317853625
app name is  https://podcasts.apple.com/us/podcast/the-carpool-with-kelly-and-lizz
podcast_id is 1606787270
app name is  https://podcasts.apple.com/us/podcast/open-and-shut-podcast
podcast_id is 1617360554
app name is  https://podcasts.apple.com/us/podcast/a-new-untold-story
podcast_id is 1538450955
app name is  https://podcasts.apple.com/us/podcast/club-random-with-bill-maher
podcast_id is 1613459129
app name is  https://podcasts.apple.com/us/podcast/in-flow
podcast_id is 1659460203
app name is  https://podcasts.apple.com/us/podcast/wildland-fire-lessons-learned-center
podcast_id is 1157695571
app name is  https://podcasts.apple.com/us/podcast/the-encounter-table
podcast_id is 1614569441
app name is  https://podcasts.apple.com/us/podcast/doubletake
podcast_id is 1634415828
app name is  https://podcasts.apple.com/us/podcast/lawfare-no-bull
podcast_id is 1585705233
app name is  https://podcasts.appl

ERROR:Base:Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url: /v1/catalog/us/apps/1676122919/reviews?l=en-GB&offset=0&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 404 error responses'))


app name is  https://podcasts.apple.com/us/podcast/the-secret-room-true-stories
podcast_id is 1082252585
app name is  https://podcasts.apple.com/us/podcast/the-doctors-farmacy-with-mark-hyman-m-d
podcast_id is 1382804627
app name is  https://podcasts.apple.com/us/podcast/market-mondays
podcast_id is 1528535502
app name is  https://podcasts.apple.com/us/podcast/the-trypod
podcast_id is 1461841525
app name is  https://podcasts.apple.com/us/podcast/labyrinths-getting-lost-with-amanda-knox
podcast_id is 1494368441
app name is  https://podcasts.apple.com/us/podcast/the-editors
podcast_id is 1150237453
app name is  https://podcasts.apple.com/us/podcast/ear-biscuits
podcast_id is 717407884
app name is  https://podcasts.apple.com/us/podcast/inner-peace-meditations
podcast_id is 1559888338
app name is  https://podcasts.apple.com/us/podcast/tom-browns-body
podcast_id is 1528040283
app name is  https://podcasts.apple.com/us/podcast/x-men-the-audio-drama
podcast_id is 902813725
app name is  https:

ERROR:Base:Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url: /v1/catalog/us/apps/1672411111/reviews?l=en-GB&offset=0&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 404 error responses'))


app name is  https://podcasts.apple.com/us/podcast/pride-of-detroit-for-detroit-lions-fans
podcast_id is 1040380210
app name is  https://podcasts.apple.com/us/podcast/rico-brogna
podcast_id is 1627097720
app name is  https://podcasts.apple.com/us/podcast/the-read
podcast_id is 619369512
app name is  https://podcasts.apple.com/us/podcast/odd-trails
podcast_id is 1598762965
app name is  https://podcasts.apple.com/us/podcast/limelight
podcast_id is 1567365630
app name is  https://podcasts.apple.com/us/podcast/operation-tradebom
podcast_id is 1647611347
app name is  https://podcasts.apple.com/us/podcast/smashing-security
podcast_id is 1195001633
app name is  https://podcasts.apple.com/us/podcast/the-bossbabe-podcast
podcast_id is 1453389736
app name is  https://podcasts.apple.com/us/podcast/the-bomb
podcast_id is 1524778767
app name is  https://podcasts.apple.com/us/podcast/darko-audio-podcast
podcast_id is 1368388920
app name is  https://podcasts.apple.com/us/podcast/the-working-genius-po

ERROR:Base:Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url: /v1/catalog/us/apps/1671005263/reviews?l=en-GB&offset=0&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 404 error responses'))


app name is  https://podcasts.apple.com/us/podcast/hotboxin-with-mike-tyson
podcast_id is 1449316560
app name is  https://podcasts.apple.com/us/podcast/tomorrows-monsters
podcast_id is 1546892635
app name is  https://podcasts.apple.com/us/podcast/real-dictators
podcast_id is 1516194847
app name is  https://podcasts.apple.com/us/podcast/adeptus-ridiculous
podcast_id is 1542416465
app name is  https://podcasts.apple.com/us/podcast/ted-talks-science-and-medicine
podcast_id is 470623801
app name is  https://podcasts.apple.com/us/podcast/business-school-with-sharran-srivatsaa
podcast_id is 1511923051
app name is  https://podcasts.apple.com/us/podcast/the-crew-has-it
podcast_id is 1622189736
app name is  https://podcasts.apple.com/us/podcast/occult-symbolism-and-pop-culture-with-isaac-weishaupt
podcast_id is 792508884
app name is  https://podcasts.apple.com/us/podcast/the-stories-we-tell-with-joanna-gaines
podcast_id is 1652798324
app name is  https://podcasts.apple.com/us/podcast/how-to-fix

In [31]:
ratings

[[{'itunes_id': '1660320068',
   'user': 'Octopig99',
   'rating': 5,
   'title': 'Excellent companion podcast',
   'review': 'The last of us podcast is an interesting look into how the show was made and thoughts from the writers about each of the episodes. There are 3 host: the 2 show writers and the actor who played the main character in the original and is also a big fan of the series. In each episode they go through the events of the corresponding episode from the TV show and cover thoughts and reaction that they had as well as writing decisions and behind the scenes information. I would highly recommend this podcast to anyone wanting to watch the show or those who have already seen it. I would recommend watching the episode first and then the corresponding podcast episode in a sort of back and forth order, but if you have already finished the show it is still very interesting to go back and listen to the whole podcast. I think that the last of us podcast is very well produced and 